In [0]:
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import Subset, DataLoader
from torchvision.models import resnet34
from google.colab import auth
from googleapiclient.discovery import build
from torchvision.datasets import ImageFolder
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Сегодня мы будем использовать transfer learning чтобы обучить классификатор героев классических звёздных войн. Эта практика требует обучения на GPU (иначе код будет выполнятся очень долго), поэтому используйте https://colab.research.google.com.

Сначала нужно включить поддержку GPU. Зайдите в Edit -> Notebook settings и как hardware accelerator укажите GPU.

Теперь нужно скачать данные, выполните следующую ячейку.
В первый раз она попросит вас перейти по ссылке - откройте эту ссылку и скопируйте оттуда токен (длинную строку в base64), вставьте этот токен в окошко под ячейкой.

In [0]:
def download_data(file_id, file_name):
  import io
  from googleapiclient.http import MediaIoBaseDownload

  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    _, done = downloader.next_chunk()
    
  downloaded.seek(0)
  with open(file_name, "wb") as f:
    f.write(downloaded.read())

  
auth.authenticate_user()
drive_service = build('drive', 'v3')

file_id = '139wA_Z9kustXy54ifhWWHJvARo5f7O6y'
file_name = 'star_wars.tar.gz'

download_data(file_id, file_name)
!tar xf star_wars.tar.gz

Важная часть обучения нейронных сетей (как и обычно в машинном обучении)  - это работа с данными. Необходимо представить данные в векторном виде и отдать их модели для обучения. В фреймворке Pytorch для этого используется связка двух классов: https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset и https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader. Первый это даже не класс, а интерфейс - у него всего два интересных метода:
__getitem__ - выбрать из датасета элемент по заданному числовому индексу
__len__ - вернуть длину датасета.

Как видите этот интерфейс эквивалентен обычному массиву. 

DataLoader в свою очередь занимается тем что выбирает элементы из Dataset который передается ему в конструкторе и делает из них батчи для обучения нейронной сети. Вы можете контролировать размер батча с помощью параметра batch_size, также не забудьте включать перемешивание данных перед каждой эпохой передавая в DataLoader параметр shuffle=True. В более продвинутых случаях возможно задавать стратегию того как именно конструируются батчи с помощью классов Sampler и BatchSampler, но мы в нашей практике этих возможностей касаться не будем.

Главное отличие Dataset от массива на практике в том что этот интерфейс не обязывает вас хранить весь набор данных в памяти. Типичная реализация по индексу подгружает данные с диска и делает из них сэмпл. 

Для задачи классификации изображений не нужно писать свой DataLoader - вместо этого есть готовый из пакета torchvision: https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.ImageFolder 
Он требует следующей структуры файлов на диске:

```
root/dog/xxx.png
root/dog/xxy.png
root/dog/xxz.png

root/cat/123.png
root/cat/nsdf3.png
root/cat/asd932_.png
```

Другими словами изображения должны быть сгруппированы по классам в папки с названиями своего класса.

In [0]:
#Попробуйте сейчас создать ImageFolder от папки star_wars которую мы сейчас скачали и вывести на экран первое изображение (это делается также как и с массивом: `data[0]`)

Изображение вывелось в полном размере. Мы собираемся использовать transfer learning - взять сеть предобученную на ImageNet и доучить её на наших изображениях. Сети тренировавшиеся на ImageNet требуют стандартного размера картинок: 224x224.
Нам нужно поресайзить картинки. У ImageFolder для этого есть аргумент конструктора transform=. Опять, нам не нужно самим писать код приведения изображения к размеру 224x224. Для этого мы воспользуемся тем-же пакетом torchvision.

In [0]:
from torchvision import transforms

augmentations = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

Здесь мы поресайзили изображение к размеру 224х224 и преобразовали его в тензор - многомерный массив, который можно далее дать на вход сети для обучения. Пересоздайте ImageFolder с данными трансформациями.

Для оценки качества нам нужна тестовая выборка. Сейчас мы поступим просто - поделим все данные в соотношении 70/30.
Чтобы это сделать можно воспользоваться классом Subset.
Он оборачивает данный в конструкторе Dataset предоставляя доступ только к элементам по переданным индексам. Его идея суть как в этом коде:
```
array = [1, 2, 3, 4, 5]

class Subset:
    def __init__(self, array, indexes):
         self.indexes = indexes
         
    def __len__(self):
         return len(self.indexes)
         
    def __getitem__(self, i):
         return array[self.indexes[i]]
         
         
 Subset(array, [0, 3, 4])[1] # Вернет 4
```

Поделите наш ImageFolder на train и test с помощью: 
- np.arange - чтобы сгенерировать индексы
- train_test_split - чтобы выбрать подмножество индексов
- Subset - чтобы выбрать подвыборку данных


In [0]:
# Разбейте data на train и test в соотношении 70/30

In [0]:
# Здесь мы берем предобученный resnet34 и заменяем в нём последний слой на голову классифицирующую изображение на 1 из 4 классов
# Берём кросс энтропию в качестве лосса и оптимизатор адам
# Мы замораживаем все слои сети кроме последнего, который будем обучать далее


model = resnet34(pretrained=True)
for param in model.parameters():
  param.requires_grad=False

criterion = torch.nn.CrossEntropyLoss()

num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 4)
model.to('cuda')

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


In [0]:
train_loader = # создайте DataLoader от train с batch_size=256. Пошаффлите данные.
test_loader = # создайте DataLoader от test с batch_size=256

In [0]:
#Эта функция считает точность модели - на вход передается сама модель, номер эпохи и тестовый лоадер.

def run_test_on_epoch(model, epoch, test_loader):
    model.eval()
    with torch.no_grad():
      test_accuracy = []
      test_real = []
      for batch_x, batch_y in tqdm(test_loader):
          outputs = model(batch_x.to('cuda')).detach().cpu().numpy()
          test_accuracy.append(outputs)
          test_real.append(batch_y.detach().cpu().numpy())
      print("Epoch", epoch, "test accuracy", accuracy_score(np.hstack(test_real), np.argmax(np.hstack(test_accuracy), axis=1)))
    model.train()

In [0]:
#Напишите код для обучения модели 25 эпох. В конце каждой эпохи вызывайте run_test_on_epoch() чтобы следить за точностью

Повторите эксперимент выше после того как добавите несколько аугментаций из https://pytorch.org/docs/stable/torchvision/transforms.html

То что стоит попробовать:
- повороты изображения до 45 градусов
- приведение в ЧБ
- ??? (то что придумаете сами)